### Spacy modellen:
https://drive.google.com/drive/folders/1AzG_0rQbi4SYRyxSno41R8wCUszkRG-r?usp=sharing

In [1]:
# Imports
import re
import spacy
import string
import json
import numpy as np
import pandas as pd
import dateparser
from datetime import datetime, timezone, timedelta
import pickle

pd.options.mode.chained_assignment = None

In [2]:
save_path = 'data/minbesluit.pkl.gz'

In [3]:
# datums als '2 1 juni 2023' omzetten naar '21 juni 2023' en double whitespace weghalen
def join_numbers(s):
    s = ' '.join(s.split())
    return re.sub(r"(\s?\d)(\s)(\d\s)", '\g<1>\g<3>', s)

In [4]:
min2023 = pd.read_pickle('data/minverzoek.pkl.gz', compression='gzip')

besluit_regex6_2 = r"\.\sBesluit\s([A-Z]*(.*?\.\s){0,4})"
besluit_regex8_2 = r"\.\sBesluit:\s([A-Z]*(.*?\.\s){0,4})"
besluit_regex9_2 = r"\d\sBesluit\s([A-Z]*(.*?\.\s){0,4})"
besluit_regex10_2 = r"\,\sBesluit\s([A-Z]*(.*?\.\s){0,4})"

def regex_context_besluit(s):
    if type(s) == float:
        return None
    else:
        search_result = re.search(besluit_regex6_2, join_numbers(s))
        if search_result == None:
            search_result = re.search(besluit_regex8_2, join_numbers(s))
        if search_result == None:
            search_result = re.search(besluit_regex9_2, join_numbers(s))
        if search_result == None:
            search_result = re.search(besluit_regex10_2, join_numbers(s))
        if search_result != None:
            result = search_result.group(1)
            return result
        else:
            return search_result

bs func def


In [5]:
# besluit context ophalen
min2023['besluit_context'] = min2023['foi_bodyTextOCR'].apply(lambda x: regex_context_besluit(x))

# opgehaalde besluiten naar alle rijen van een document uitsmeren
min2023['besluit_context'] = min2023['besluit_context'].groupby(level=0).fillna(method='ffill')
min2023['besluit_context'] = min2023['besluit_context'].groupby(level=0).fillna(method='bfill')

In [6]:
nlp_gescheiden = spacy.load("data/spacy_models/nl_besluit_gescheiden_lg")

def type_besluit(row):
    if row.name[1] == 1:
        besluit_text = row['besluit_context']
        if besluit_text != None:
            doc = nlp_gescheiden(besluit_text)
            hoogste = max(doc.cats, key=doc.cats.get)
            perc = doc.cats[hoogste]
            return hoogste, round(perc*100, 2)
        else: 
            return None, None

min2023['besluit_spacy'] = min2023.apply(lambda x: type_besluit(x), axis=1)
min2023[['besluit_type', 'besluit_zekerheid']] = pd.DataFrame(min2023['besluit_spacy'].tolist(), index=min2023.index)
min2023.drop('besluit_spacy', axis=1, inplace=True)

min2023['besluit_type'] = min2023['besluit_type'].groupby(level=0).fillna(method='ffill')
min2023['besluit_type'] = min2023['besluit_type'].groupby(level=0).fillna(method='bfill')
min2023['besluit_zekerheid'] = min2023['besluit_zekerheid'].groupby(level=0).fillna(method='ffill')
min2023['besluit_zekerheid'] = min2023['besluit_zekerheid'].groupby(level=0).fillna(method='bfill')

In [9]:
min2023['besluit_type'].value_counts()

besluit_type
deels openbaar    15969
afgewezen          7558
openbaar           5316
niet aanwezig       559
Name: count, dtype: int64

In [ ]:
min2023.to_pickle(save_path, compression='gzip')